In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import random as rn
from keras import Input
#from keras.engine import Model
from keras.layers import Dense, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, Multiply, Conv2D, MaxPooling2D
import librosa
import pandas as pd
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers

In [2]:
volume_path = 'C:/Users/USER/Desktop/spcup_2022_training_part1/spcup_2022_training_part1'

FilePathList = []

for dirName, subdirList, fileList in os.walk(volume_path):
    for filename in fileList:
        print(filename)
        if '.csv' not in filename:
            #print(filename)
            FilePathList.append(volume_path + '/' + filename)
        else:
            pd_label = pd.read_csv(volume_path + '/' + filename)
            print(pd_label)

labels = np.array(pd_label['algorithm'])

## prepare unknown data
volume_path_unknown = 'C:/Users/USER/Desktop/spcup_2022_unseen/spcup_2022_unseen'

FilePathList_unknown = []

for dirName, subdirList, fileList in os.walk(volume_path_unknown):
    for filename in fileList:
        print(filename)
        if '.csv' not in filename:
            #print(filename)
            FilePathList_unknown.append(volume_path_unknown + '/' + filename)
        else:
            pd_label = pd.read_csv(volume_path_unknown + '/' + filename)
            print(pd_label)

labels_unknown = np.array(pd_label['algorithm'])

00050dd7458cf08e594c797930696bce.wav
00070e7c531000d3dddc735d107275a9.wav
000f0711027a69b7f3886c2dbcb7d41f.wav
001e28e66dee24408aaf3480dfb95fbe.wav
001eee950f60613869544b72cd48fe97.wav
001f59a6ef884e2071ba1ee52cba8d09.wav
0022a32237a11ee475c7b96d82369596.wav
0022e7014db5482004d9e6f731cfbc8b.wav
00317a6d7effa97c0dc7e1cf1b6d542d.wav
0036636e89ad53945f0f2707c09f148e.wav
004e173fc8d58405c9cec8b256f52842.wav
006c78a2629d193baf69107e6f9557ff.wav
0081bef2286c79fd30cae67792086100.wav
00856bb9827f0d85111fa409211dbf7e.wav
008e269e45cb63d2e6f07508a5482000.wav
0093f56df1acd069bbf88ede88eb528d.wav
00a0661d8351e5132cb0df5e1b4f19e2.wav
00aa20bc49a1008240105ad2d63a0f64.wav
00afd54110183acd564d14422119da23.wav
00b75d43ebb31451f80e742fdab790d7.wav
00c04d4ba166de6330f874014e58f6c6.wav
00c9a92b0acc342402454e6db238bbcc.wav
00d4474165837abbefaf1ae62bfa67e8.wav
00d4b9d83e9fffc2ac59549c8219b8eb.wav
00dd0e930b7e6dc5a9f2286127fac6e1.wav
00dd2d99dfbc84e476a772579261bbcd.wav
00f1ff1f3459060f3063c5646b4f2735.wav
0

In [3]:
FilePathList.extend(FilePathList_unknown)

In [4]:
FilePathList_6000 = FilePathList

In [5]:
label_6000 = np.concatenate((labels,labels_unknown),axis=0)

In [6]:
len(FilePathList_6000)

6000

In [7]:
label_6000.shape

(6000,)

In [8]:
##파일마다 1초씩 슬라이싱 해서 data augmentation + 마지막에 파일 별로 score 계산하기 위한 preprocess
# X_aug = []

sampling_rate = 16000   # 1초에 16000번
n_mels = 64
test_accuracy_all = []

X_train_path, X_test_path, y_train_raw, y_test_raw = train_test_split(np.array(FilePathList_6000), label_6000, test_size=0.2, stratify=label_6000, random_state=42)

In [9]:
##train set
print('raw train_set_num :', len(y_train_raw))

y_labels_aug = []
X_mel_aug = []
for i, filepath in enumerate(X_train_path):
    y, sr = librosa.load(filepath, mono=True, sr=sampling_rate)
    index_f = 0
    for j in range(y.size // sampling_rate):
        X_aug_seg = y[index_f:index_f + sampling_rate]
        
        S = librosa.feature.melspectrogram(y=X_aug_seg, sr=sampling_rate, n_mels=n_mels, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)
        X_mel_aug.append(S_dB)

        y_labels_aug.append(y_train_raw[i])

        index_f = index_f + sampling_rate
        
X_train1 = np.array(X_mel_aug)
y_train1 = np.array(y_labels_aug)
print('train set.shape : ', X_train1.shape)
print('y_train.shape', y_train1.shape)

raw train_set_num : 4800
train set.shape :  (31018, 64, 32)
y_train.shape (31018,)


In [10]:
##test set
print('raw test_set_num :', len(y_test_raw))
y_labels_aug = []
X_mel_aug = []
file_split_num = []
for i, filepath in enumerate(X_test_path):
    y, sr = librosa.load(filepath, mono=True, sr=sampling_rate)
    index_f = 0
    for j in range(y.size // sampling_rate):
        X_aug_seg = y[index_f:index_f + sampling_rate]

        S = librosa.feature.melspectrogram(y=X_aug_seg, sr=sampling_rate, n_mels=n_mels, fmax=8000)
        #S = librosa.feature.melspectrogram(y=X_aug_seg, sr=sampling_rate, n_fft=512, hop_length=512, n_mels=n_mels, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)
        X_mel_aug.append(S_dB)

        y_labels_aug.append(y_test_raw[i])

        index_f = index_f + sampling_rate
    file_split_num.append(j + 1)

X_test1 = np.array(X_mel_aug)
y_test1 = np.array(y_labels_aug)
print('splited_test set.shape : ', X_test1.shape)
print('y_test.shape : ', y_test1.shape)
print(len(file_split_num))

raw test_set_num : 1200
splited_test set.shape :  (7558, 64, 32)
y_test.shape :  (7558,)
1200


In [11]:
# Data Reshape

print('np.unique(y_train)', np.unique(y_train1))
print('np.unique(y_test)', np.unique(y_test1))

X_train = X_train1.reshape(X_train1.shape[0], X_train1.shape[1], X_train1.shape[2], 1)
X_test = X_test1.reshape(X_test1.shape[0], X_test1.shape[1], X_test1.shape[2], 1)

print('X_train.shape : ', X_train.shape)
print('X_test.shape : ', X_test.shape)

y_train = y_train1
y_test = y_test1

print('y_train.shape', y_train.shape)
print('y_test.shape : ', y_test.shape)

np.unique(y_train) [0 1 2 3 4 5]
np.unique(y_test) [0 1 2 3 4 5]
X_train.shape :  (31018, 64, 32, 1)
X_test.shape :  (7558, 64, 32, 1)
y_train.shape (31018,)
y_test.shape :  (7558,)


In [12]:
import tensorflow as tf

model = tf.keras.models.load_model('C:/Users/USER/Desktop/best_model/42_spcup_best (2).h5')

In [13]:
prediction = model.predict(X_test)

In [14]:
from scipy.stats import mode

predicted_classes = np.argmax(prediction, axis=1)
f = 0
test_predict = []
softmax_val = []

for j,i in enumerate(file_split_num):
    print(y_test_raw[j])
    print(prediction[f:f + i])
    print(predicted_classes[f:f + i])
    print('\n')
    softmax_val.append(prediction[f:f + i])
    test_predict.append(mode(predicted_classes[f:f + i])[0][0])
    f += i

3
[[1.2368822e-13 4.2546362e-02 9.0892223e-07 9.5745152e-01 1.3086172e-06]
 [2.1576419e-15 3.4581672e-07 1.8699480e-09 9.9999964e-01 2.3442146e-09]
 [3.8312773e-11 6.0681657e-07 2.2785056e-08 9.9999940e-01 1.8969092e-11]
 [1.1625479e-09 2.8561570e-07 6.3339121e-07 9.9999917e-01 3.7902796e-09]
 [9.0506114e-15 6.5926015e-06 6.7022398e-08 9.9999332e-01 5.7887268e-14]
 [9.6222552e-10 6.4485798e-06 3.1422953e-06 9.9999046e-01 8.4171725e-11]
 [2.2708356e-10 4.0909112e-03 6.1507910e-07 9.9590826e-01 2.2653752e-07]
 [4.0986457e-11 2.3808279e-03 1.5623760e-07 9.9761909e-01 7.4489637e-10]]
[3 3 3 3 3 3 3 3]


3
[[8.0553278e-16 5.1520495e-08 3.1389644e-10 1.0000000e+00 5.2921772e-14]
 [9.4264338e-17 3.8743039e-08 1.5118009e-11 1.0000000e+00 2.9748972e-12]
 [2.7939082e-18 2.1692652e-09 9.6359925e-14 1.0000000e+00 4.5104708e-14]
 [1.0838729e-11 8.8697545e-02 3.9868470e-04 9.1090369e-01 4.6619775e-10]
 [1.2404926e-13 2.3631706e-08 1.5785195e-08 1.0000000e+00 4.2478282e-14]
 [4.8273742e-15 2.3133588e

In [15]:
len(test_predict)

1200

In [16]:
acc_bool = test_predict == y_test_raw
test_accuracy = acc_bool.tolist().count(True)/len(acc_bool)
print('test_accuracy: ',test_accuracy)

test_accuracy:  0.83


In [34]:
softmax_val[2]

array([[1.2229421e-07, 2.3557205e-02, 9.7601527e-01, 4.2750532e-04,
        6.6425365e-10],
       [6.5581478e-09, 1.9622308e-03, 9.9803764e-01, 1.7229360e-07,
        1.2567658e-13],
       [1.3654242e-08, 1.8960639e-04, 9.9981040e-01, 1.1709622e-09,
        1.3364414e-10]], dtype=float32)

In [33]:
softmax_val[2][:,0]

array([1.2229421e-07, 6.5581478e-09, 1.3654242e-08], dtype=float32)

In [60]:
mean = []

In [61]:
for i in range(1200):
    for j in range(5):
            temp.append(softmax_val[i][:,j].mean())
    temp = np.array(temp)
    mean.append(temp)
    temp = []        
    

In [65]:
mean

[array([3.0391140e-10, 6.1290478e-03, 6.9345037e-07, 9.9387014e-01,
        1.9276717e-07], dtype=float32),
 array([1.1751908e-12, 8.8717919e-03, 3.9875998e-05, 9.9108839e-01,
        5.5024173e-11], dtype=float32),
 array([4.7502201e-08, 8.5696811e-03, 9.9128777e-01, 1.4255960e-04,
        2.6600783e-10], dtype=float32),
 array([9.9999946e-01, 9.6258330e-18, 5.5590283e-07, 5.5875270e-23,
        2.5891173e-13], dtype=float32),
 array([1.1904996e-09, 9.9125963e-01, 8.7332753e-03, 7.0466845e-06,
        3.7053327e-10], dtype=float32),
 array([1.9656449e-05, 5.1443744e-01, 4.8109573e-01, 4.3099443e-03,
        1.3723171e-04], dtype=float32),
 array([4.4778938e-07, 2.1166497e-06, 5.9741815e-06, 4.0114250e-07,
        9.9999112e-01], dtype=float32),
 array([7.9613392e-06, 5.3854930e-01, 3.7396857e-01, 8.7456129e-02,
        1.7983217e-05], dtype=float32),
 array([1.4958162e-03, 4.5880158e-02, 9.5256370e-01, 5.4626242e-05,
        5.7123566e-06], dtype=float32),
 array([2.1311818e-08, 4.588